In [27]:
import easydict
import os
import time

from utils.utils import *
import torch
import torch.utils.data
import torch.utils.tensorboard
#import tqdm

ModuleNotFoundError: No module named 'utils.utils'

In [26]:
# 다양한 입력 매개변수를 분석한다. 아니면 기본 매개변수를 사용한다
# https://worthpreading.tistory.com/56
args = easydict.EasyDict({
    "epoch": 20,
    "gradient_accumulation": 1,
    "multiscale_training": True,
    "batch_size": 32,
    "num_workers": 8,
    "data_config": "config/voc.data",
    "pretrained_weights": 'weights/darknet53.conv.74',
    "image_size": 416
})
print(args)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
now = time.strftime('%y%m%d_%H%M%S', time.localtime(time.time()))

# Tensorboard writer 객체 생성
log_dir = os.path.join('logs', now)
os.makedirs(log_dir, exist_ok=True)
writer = torch.utils.tensorboard.SummaryWriter(log_dir)

# 데이터셋 설정값을 가져오기
data_config = utils.utils.parse_data_config(args.data_config)
train_path = data_config['train']
valid_path = data_config['valid']
num_classes = int(data_config['classes'])
class_names = utils.utils.load_classes(data_config['names'])


{'epoch': 20, 'gradient_accumulation': 1, 'multiscale_training': True, 'batch_size': 32, 'num_workers': 8, 'data_config': 'config/voc.data', 'pretrained_weights': 'weights/darknet53.conv.74', 'image_size': 416}


AttributeError: module 'utils' has no attribute 'utils'